# 1.Import Library

In [2]:
import os
import pandas as pd
import numpy as np
import pyodbc

# 2. Connect to SQL Server

In [3]:
conn = pyodbc.connect( #CONNECTION
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=LAPTOP-OI1U6J19\MSSQLSERVER01;"
    "DATABASE=VTI;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;")

df = pd.read_sql_query(  #Data Frame
'''                                        
SELECT *
FROM [VTI].[dbo].[Airline_pre];                         
''', conn)

df

<>:3: SyntaxWarning: "\M" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\M"? A raw string is also an option.
<>:3: SyntaxWarning: "\M" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\M"? A raw string is also an option.
C:\Users\Bom's PC\AppData\Local\Temp\ipykernel_25032\2934623139.py:3: SyntaxWarning: "\M" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\M"? A raw string is also an option.
  "SERVER=LAPTOP-OI1U6J19\MSSQLSERVER01;"
C:\Users\Bom's PC\AppData\Local\Temp\ipykernel_25032\2934623139.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(  #Data Frame


,Passenger_ID,First_Name,Last_Name,Gender,Age,Nationality,Airport_Name,Airport_Country_Code,Country_Name,Airport_Continent,Continents,Departure_Date,Arrival_Airport,Pilot_Name,Flight_Status,Today
0,zQtk5b,Arda,Molson,Female,24,United States,Maria Reiche Neuman Airport,PE,Peru,SAM,South America,2022-07-25,NZC,Patrizia Ciccarello,On Time,2026-01-24
1,3rDKTc,Paquito,Walicki,Male,50,China,Mbuji Mayi Airport,CD,"Congo, The Democratic Republic of the",AF,Africa,2022-06-04,MJM,Buddy Pascho,Delayed,2026-01-24
2,0z876q,Cobbie,Treadgall,Male,58,Poland,Kalkgurung Airport,AU,Australia,OC,Oceania,2022-07-04,KFG,Carmelle Sposito,On Time,2026-01-24
3,wk9ZwM,Maryjane,Trosdall,Female,40,Poland,Binaka Airport,ID,Indonesia,AS,Asia,2022-07-11,GNS,Carmine Cookney,Delayed,2026-01-24
4,6coA01,Raine,Skittreal,Female,32,Nigeria,Rajiv Gandhi International Airport,IN,India,AS,Asia,2022-07-31,HYD,Cassandry McRobert,Cancelled,2026-01-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98614,d5o2xU,Ric,Laroux,Male,61,Lithuania,Immokalee Regional Airport,US,United States,NAM,North America,2022-12-17,IMM,Harwilll Neale,Delayed,2026-01-24
98615,iz6cxd,Wilfrid,Nibley,Male,56,Russia,Erzurum International Airport,TR,Turkey,AS,Asia,2022-12-08,ERZ,Harris Cockshott,On Time,2026-01-24
98616,x4693Z,Josi,Sanger,Female,45,Ireland,Bahías de Huatulco International Airport,MX,Mexico,NAM,North America,2022-05-16,HUX,Tansy Beamont,Cancelled,2026-01-24
98617,6NhZJF,Karita,Petrusch,Female,28,Russia,Marana Regional Airport,US,United States,NAM,North America,2022-10-07,AVW,Smith Kenlin,Delayed,2026-01-24


# 3. Preprocessing Data

## 3.1. General Information

In [4]:
df.shape

(98619, 16)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98619 entries, 0 to 98618
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Passenger_ID          98619 non-null  object
 1   First_Name            98619 non-null  object
 2   Last_Name             98619 non-null  object
 3   Gender                98619 non-null  object
 4   Age                   98619 non-null  int64 
 5   Nationality           98619 non-null  object
 6   Airport_Name          98619 non-null  object
 7   Airport_Country_Code  98619 non-null  object
 8   Country_Name          98619 non-null  object
 9   Airport_Continent     98619 non-null  object
 10  Continents            98619 non-null  object
 11  Departure_Date        98619 non-null  object
 12  Arrival_Airport       98619 non-null  object
 13  Pilot_Name            98619 non-null  object
 14  Flight_Status         98619 non-null  object
 15  Today                 98619 non-null

In [6]:
df.describe()

,Age
count,98619.000000
mean,45.504021
std,25.929849
min,1.000000
25%,23.000000
50%,46.000000
75%,68.000000
max,90.000000


## 3.2. Detect and Remove Outliers

In [7]:
import pandas as pd
import numpy as np

def detect_outliers_iqr(df):
    outlier_summary = {}

    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]

        outlier_summary[col] = {
            "lower_bound": lower_bound,
            "upper_bound": upper_bound,
            "outlier_count": outliers.shape[0],
            "outlier_index": outliers.index.tolist()
        }

    return outlier_summary
outliers_iqr = detect_outliers_iqr(df)

for col, info in outliers_iqr.items():
    print(f"{col}: {info['outlier_count']} outliers")

Age: 0 outliers


## 3.3. Remove Duplicate Rows

In [8]:
df.duplicated().sum()

np.int64(0)

## ADDING USERFUL INFORMATION 

In [9]:
import pandas as pd

# =========================
# 1. Load dữ liệu OurAirports (online)
# =========================
airports_url = "https://ourairports.com/data/airports.csv"
countries_url = "https://ourairports.com/data/countries.csv"

airports = pd.read_csv(airports_url)
countries = pd.read_csv(countries_url)

# =========================
# 2. Chuẩn hóa dữ liệu country
# =========================
# code: ISO-2 (VN, JP, US)
# name: Vietnam, Japan, United States
country_map = countries.set_index("code")["name"]

# =========================
# 3. Tạo mapping Airport -> Country Name
# =========================
airport_country_map = (
    airports
    .dropna(subset=["iata_code"])
    .assign(country_name=lambda x: x["iso_country"].map(country_map))
    .set_index("iata_code")["country_name"]
)

# =========================
# 4. Chuẩn hóa airport code trong dữ liệu của bạn
# =========================
df["Arrival_Airport"] = (
    df["Arrival_Airport"]
    .astype(str)
    .str.strip()
    .str.upper()
)

# =========================
# 5. Map vào DataFrame chính
# =========================
df["Arrival_Country"] = df["Arrival_Airport"].map(airport_country_map)

# =========================
# 6. (Optional) xử lý airport không map được
# =========================
df["Arrival_Country"].fillna("Unknown", inplace=True)

# =========================
# 7. Kiểm tra kết quả
# =========================
print(df)

      Passenger_ID First_Name  Last_Name  Gender  Age    Nationality  \
0           zQtk5b       Arda     Molson  Female   24  United States   
1           3rDKTc    Paquito    Walicki    Male   50          China   
2           0z876q     Cobbie  Treadgall    Male   58         Poland   
3           wk9ZwM   Maryjane   Trosdall  Female   40         Poland   
4           6coA01      Raine  Skittreal  Female   32        Nigeria   
...            ...        ...        ...     ...  ...            ...   
98614       d5o2xU        Ric     Laroux    Male   61      Lithuania   
98615       iz6cxd    Wilfrid     Nibley    Male   56         Russia   
98616       x4693Z       Josi     Sanger  Female   45        Ireland   
98617       6NhZJF     Karita   Petrusch  Female   28         Russia   
98618       nESe6U      Goldy   Renfield  Female   14          China   

                                   Airport_Name Airport_Country_Code  \
0                   Maria Reiche Neuman Airport                

In [10]:
cat_cols = df.select_dtypes(include=["object", "category"]).columns
cat_cols = [c for c in cat_cols if c != "Passenger_ID"]

output_path = "categorical_summary_all.txt"

with open(output_path, "w", encoding="utf-8") as f:
    f.write("CATEGORICAL VARIABLE SUMMARY (Grouped by Passenger_ID)\n")
    f.write("=" * 60 + "\n")

    for col in cat_cols:
        distinct_cnt = df[col].nunique(dropna=True)

        if distinct_cnt < 20:
            f.write(f"\n=== {col} (distinct={distinct_cnt}) ===\n")
            counts = (
                df.groupby(col)["Passenger_ID"]
                  .nunique()
                  .sort_values(ascending=False)
            )

            for category, cnt in counts.items():
                f.write(f"{category}: {cnt}\n")
        else:
            f.write(f"\n--- SKIPPED {col} (distinct={distinct_cnt}) ---\n")

#Insights Cateogryrical Variables But it is hard explain here

In [ ]:
from openai import OpenAI

with open("categorical_summary_all.txt", "r", encoding="utf-8") as f:
    txt_content = f.read()

client = OpenAI(
    api_key=""
)

response = client.responses.create(
    model="gpt-4.1-mini",
    input=[
        {
            "role": "user",
            "content": f"""
Analyze the following categorical respondent summary.
Give 5 key insights, anomalies, and high-level observations.

TEXT:
{txt_content}
"""}
    ]
)

print(response.output_text)

# 4. Exploration Data Analytics (EDA)

In [ ]:
# Biz need
     #--Người dân nước nào có xu hướng di chuyển bằng máy bay nhiều nhất?
     #--Airport nào có tình trạng cancelled và delay nhiều nhất?
     #--Nước nào được đi đến nhiều nhiều nhất?
from openai import OpenAI

with open("categorical_summary_all.txt", "r", encoding="utf-8") as f:
    txt_content = f.read()

client = OpenAI(
    api_key=""
)

response = client.responses.create(
    model="gpt-4.1-mini",
    input=[
        {
            "role": "user",
            "content": f"""

    #--Người dân nước nào có xu hướng di chuyển bằng máy bay nhiều nhất? #xuhuongdichuyen

    #--Airport nào có tình trạng cancelled và delay nhiều nhất? #tinhtrangairport

    #--Nước nào được đi đến nhiều nhiều nhất? #nuocduocdennhieu

Analyze the following schema summary. Answers which columns are relevant to answer the above business needs from txt_content.

Return it ### Summary of Relevant Columns by Business Needs:

TEXT:
{txt_content}
"""}
    ]
)

print(response.output_text)

### Support us to read huge schema text and give relevant columns:

In [11]:
# 1. #--Người dân nước nào có xu hướng di chuyển bằng máy bay nhiều nhất? #xuhuongdichuyen**
# Relevant Columns: "Nationality", "Passenger_ID"

from collections import Counter

language_counter = Counter()

for langs in df["Nationality"].dropna():
    for lang in langs.split(";"):
        language_counter[lang.strip()] += 1

# Convert to DataFrame if needed
lang_count_df = (
    pd.DataFrame(language_counter.items(), columns=["Nationality", "Appear_Count"])
    .sort_values("Appear_Count", ascending=False)
)
lang_count_df

# We observe that China, Indonesia, Russia are top 3 Foreigner go by plane.

,Nationality,Appear_Count
1,China,18317
6,Indonesia,10559
8,Russia,5693
7,Philippines,5239
20,Brazil,3791
...,...,...
228,Cook Islands,2
238,Saint Helena,2
145,Jersey,1
237,Sint Maarten,1


In [12]:
# 2. #--Airport nào có tình trạng cancelled và delay nhiều nhất? #tinhtrangairport**
# Relevant Columns: "Airport_Name", "Flight_Status"
import pandas as pd
from collections import Counter

# ==============================
# 1. Chuẩn hóa & explode dữ liệu
# ==============================

df_clean = df[["Airport_Name", "Flight_Status"]].dropna().copy()

# Tách nhiều status nếu có dấu ;
df_clean["Flight_Status"] = (
    df_clean["Flight_Status"]
    .astype(str)
    .str.split(";")
)

df_exploded = df_clean.explode("Flight_Status")

df_exploded["Flight_Status"] = df_exploded["Flight_Status"].str.strip()

# ==============================
# 2. Đếm số lần xuất hiện (tổng quát)
# ==============================

status_count_df = (
    df_exploded
    .groupby(["Airport_Name", "Flight_Status"])
    .size()
    .reset_index(name="Appear_Count")
)

# ==============================
# 3. Top 3 airport CANCELLED nhiều nhất
# ==============================

top_cancelled_airports = (
    status_count_df[
        status_count_df["Flight_Status"].str.lower() == "cancelled"
    ]
    .sort_values("Appear_Count", ascending=False)
    .head(3)
)

# ==============================
# 4. Top 3 airport DELAY nhiều nhất
# ==============================

top_delayed_airports = (
    status_count_df[
        status_count_df["Flight_Status"].str.lower().isin(["delayed", "delay"])
    ]
    .sort_values("Appear_Count", ascending=False)
    .head(3)
)

# ==============================
# 5. Kết quả
# ==============================

print("Top 3 Airport Cancelled nhiều nhất:")
display(top_cancelled_airports)

print("Top 3 Airport Delayed nhiều nhất:")
display(top_delayed_airports)

Top 3 Airport Cancelled nhiều nhất:


,Airport_Name,Flight_Status,Appear_Count
25905,Yangzhou Taizhou Airport,Cancelled,13
13982,Mae Hong Son Airport,Cancelled,13
11996,Kiunga Airport,Cancelled,13


Top 3 Airport Delayed nhiều nhất:


,Airport_Name,Flight_Status,Appear_Count
20803,Santa Maria Airport,Delayed,17
20818,Santa Rosa Airport,Delayed,16
20648,San Pedro Airport,Delayed,15


In [13]:
# 3. #--#--Nước nào được đi đến nhiều nhiều nhất? #nuocduocdennhieu**
# Relevant Columns: "Arrival_Country", "Passenger_ID"

from collections import Counter

language_counter = Counter()

for langs in df["Arrival_Country"].dropna():
    for lang in langs.split(";"):
        language_counter[lang.strip()] += 1

# Convert to DataFrame if needed
lang_count_df2 = (
    pd.DataFrame(language_counter.items(), columns=["Arrival_Country", "Appear_Count"])
    .sort_values("Appear_Count", ascending=False)
)
lang_count_df2.head(10)

# We observe that United States, Australia and Canada are top 3 Nations that Foreigners usually arrive.

,Arrival_Country,Appear_Count
5,United States,21287
2,Australia,6265
28,Unknown,5804
12,Canada,5172
18,Papua New Guinea,3614
10,Brazil,3330
6,China,2865
3,Indonesia,2225
33,Russia,2175
13,Colombia,1607


# 5. Machine Learning

In [14]:
##  Dự đoán Flight có bị Delay / Cancelled hay không

## Flight_Status -> chuyển thành:

## 0 = On Time
## 1 = Delayed / Cancelled
##=============================
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# =========================
# 1. Chọn feature & target
# =========================

df_ml = df.copy()

# Binary target: Delay / Cancelled = 1, On Time = 0
df_ml["Is_Delayed"] = df_ml["Flight_Status"].str.lower().isin(
    ["delayed", "cancelled"]
).astype(int)

X = df_ml[
    ["Age", "Gender", "Nationality", "Airport_Continent", "Arrival_Country"]
]
y = df_ml["Is_Delayed"]

# =========================
# 2. Tách numerical / categorical
# =========================

numeric_features = ["Age"]
categorical_features = [
    "Gender",
    "Nationality",
    "Airport_Continent",
    "Arrival_Country"
]

# =========================
# 3. Preprocessing pipeline
# =========================

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)

# =========================
# 4. Model
# =========================

model = LogisticRegression(max_iter=1000)

# =========================
# 5. Full pipeline
# =========================

pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", model)
])

# =========================
# 6. Train / Test split
# =========================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# =========================
# 7. Train model
# =========================

pipeline.fit(X_train, y_train)

# =========================
# 8. Predict & Evaluate
# =========================

y_pred = pipeline.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[   14  9840]
 [   32 19700]]

Classification Report:
              precision    recall  f1-score   support

           0       0.30      0.00      0.00      9854
           1       0.67      1.00      0.80     19732

    accuracy                           0.67     29586
   macro avg       0.49      0.50      0.40     29586
weighted avg       0.55      0.67      0.53     29586



# 6. RFM Score

In [15]:
import pandas as pd

# =========================
# 1. Chuẩn hóa date
# =========================

df_rfm = df.copy()

df_rfm["Departure_Date"] = pd.to_datetime(df_rfm["Departure_Date"])
df_rfm["Today"] = pd.to_datetime(df_rfm["Today"])

# =========================
# 2. Tính R & F
# =========================

rfm = (
    df_rfm
    .groupby("Passenger_ID")
    .agg(
        Recency=("Departure_Date", lambda x: (df_rfm["Today"].max() - x.max()).days),
        Frequency=("Passenger_ID", "count")
    )
    .reset_index()
)

# =========================
# 3. Monetary (proxy)
# =========================

rfm["Monetary"] = rfm["Frequency"]

# =========================
# 4. Chia điểm RFM (1–5)
# =========================

rfm["R_Score"] = pd.qcut(rfm["Recency"], 5, labels=[5,4,3,2,1])
rfm["F_Score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm["M_Score"] = pd.qcut(rfm["Monetary"].rank(method="first"), 5, labels=[1,2,3,4,5])

# =========================
# 5. RFM Segment
# =========================

rfm["RFM_Score"] = (
    rfm["R_Score"].astype(str) +
    rfm["F_Score"].astype(str) +
    rfm["M_Score"].astype(str)
)

rfm

,Passenger_ID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
0,002Gd9,1338,1,1,3,1,1,311
1,00638T,1173,1,1,5,1,1,511
2,006RvT,1215,1,1,4,1,1,411
3,008AJe,1318,1,1,3,1,1,311
4,00Ahsk,1355,1,1,2,1,1,211
...,...,...,...,...,...,...,...,...
98614,zzmvXb,1402,1,1,2,5,5,255
98615,zzsIdD,1461,1,1,1,5,5,155
98616,zzt3WK,1412,1,1,2,5,5,255
98617,zzuv5J,1233,1,1,4,5,5,455


# 7. Save Clear Data Into Database To Visiualization

In [ ]:
import pyodbc
import pandas as pd

# =========================
# 1. Connect SQL Server
# =========================

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=LAPTOP-OI1U6J19\\MSSQLSERVER01;"
    "DATABASE=VTI;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)
cursor = conn.cursor()

# =========================
# 2. Pandas dtype → SQL type
# =========================

def dtype_map(dt):
    if pd.api.types.is_integer_dtype(dt):
        return "INT"
    elif pd.api.types.is_float_dtype(dt):
        return "FLOAT"
    elif pd.api.types.is_bool_dtype(dt):
        return "BIT"
    elif pd.api.types.is_datetime64_any_dtype(dt):
        return "DATETIME"
    else:
        return "VARCHAR(255)"

# =========================
# 3. Function tạo table
# =========================

def create_table_from_df(df, table_name):
    
    cursor.execute(f"""
        IF OBJECT_ID('dbo.{table_name}', 'U') IS NOT NULL
            DROP TABLE dbo.{table_name}
    """)

    cols = ", ".join(
        [f"[{c}] {dtype_map(df[c].dtype)}" for c in df.columns]
    )

    cursor.execute(f"CREATE TABLE dbo.{table_name} ({cols})")

    cursor.fast_executemany = True
    cursor.executemany(
        f"INSERT INTO dbo.{table_name} VALUES ({','.join(['?'] * len(df.columns))})",
        df.values.tolist()
    )

    conn.commit()
    print(f"✔ Table {table_name} created ({len(df)} rows)")

# =========================
# 4. Tạo nhiều bảng
# =========================

tables = {
    "Airline_Clean": df,
    "Airline_rfm_result": rfm,
    "Airline_delay_summary": top_delayed_airports,
    "Airline_cancel_summary": top_cancelled_airports,
    "Airport_xuhuongdichuyen":lang_count_df,
    "Airport_nuocduocdennhieu":lang_count_df2
}

for table_name, dataframe in tables.items():
    create_table_from_df(dataframe, table_name)